## Guided project under course Using Sqlite and Pandas on Large Data

- Analyzing Startup Fundraising Deals from Crunchbase 
- dataquest.com Course Mission 167

In [ ]:
import pandas as pd

In [5]:
fundraising_iter = pd.read_csv('crunchbase-investments.csv',chunksize=5000,encoding='latin-1')

total_mem_fp = 0
missing_dicts = {}
for chunk in fundraising_iter:
    #print(chunk.columns)
    #missing value counts of each column
    #print(chunk.isnull().sum().to_dict())
    missing_dicts.update(chunk.isnull().sum().to_dict())
    #mem footprint
    print(chunk.memory_usage(deep=True).sum())
    total_mem_fp += chunk.memory_usage(deep=True).sum()
    # each chunk consumes about 6mb in memory
    

5850258
5796772
5803921
5796739
5792696
5823207
5800132
5777296
5658251
4864866
2793107


In [6]:
missing_dicts.keys()

dict_keys(['company_permalink', 'company_name', 'company_category_code', 'company_country_code', 'company_state_code', 'company_region', 'company_city', 'investor_permalink', 'investor_name', 'investor_category_code', 'investor_country_code', 'investor_state_code', 'investor_region', 'investor_city', 'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter', 'funded_year', 'raised_amount_usd'])

In [7]:
no_missing_cols =[k for k,v in missing_dicts.items() if v == 0]
no_missing_cols

['company_permalink',
 'company_name',
 'company_country_code',
 'company_region',
 'investor_permalink',
 'investor_name',
 'investor_region',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year']

In [8]:
total_mem_fp

59757245

In [9]:
#which columns to drop?
chunk.describe()

,investor_category_code,investor_country_code,investor_state_code,investor_city,funded_year,raised_amount_usd
count,0.0,0.0,0.0,0.0,2870.000000,2.580000e+03
mean,NaN,NaN,NaN,NaN,2010.478746,3.799586e+06
std,NaN,NaN,NaN,NaN,2.278470,1.022031e+07
min,NaN,NaN,NaN,NaN,1998.000000,3.000000e+03
25%,NaN,NaN,NaN,NaN,2009.000000,7.500000e+05
50%,NaN,NaN,NaN,NaN,2011.000000,1.375000e+06
75%,NaN,NaN,NaN,NaN,2012.000000,3.000000e+06
max,NaN,NaN,NaN,NaN,2013.000000,2.750000e+08


In [11]:
test_df = pd.read_csv('crunchbase-investments.csv',nrows=10,encoding='latin-1')

In [12]:
test_df.isnull().sum()

company_permalink         0
company_name              0
company_category_code     0
company_country_code      0
company_state_code        1
company_region            0
company_city              1
investor_permalink        0
investor_name             0
investor_category_code    3
investor_country_code     2
investor_state_code       2
investor_region           0
investor_city             2
funding_round_type        0
funded_at                 0
funded_month              0
funded_quarter            0
funded_year               0
raised_amount_usd         0
dtype: int64

In [13]:
test_df['investor_category_code'].value_counts() #only 1 value -- finance

finance    7
Name: investor_category_code, dtype: int64

In [14]:
test_df['investor_country_code'].value_counts()

USA    8
Name: investor_country_code, dtype: int64

In [15]:
##based on test_df, these 3 columns should be dropped
for col in test_df.columns:
    if test_df[col].nunique() == 1:
        print(col)

company_country_code
investor_category_code
investor_country_code


In [47]:
drop_cols = ['company_country_code',
'investor_category_code','investor_country_code']

In [49]:
keep_cols = test_df.columns.drop(drop_cols)

In [50]:
keep_cols

Index(['company_permalink', 'company_name', 'company_category_code',
       'company_state_code', 'company_region', 'company_city',
       'investor_permalink', 'investor_name', 'investor_state_code',
       'investor_region', 'investor_city', 'funding_round_type', 'funded_at',
       'funded_month', 'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')

In [16]:
#check column dtypes
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   company_permalink       10 non-null     object
 1   company_name            10 non-null     object
 2   company_category_code   10 non-null     object
 3   company_country_code    10 non-null     object
 4   company_state_code      9 non-null      object
 5   company_region          10 non-null     object
 6   company_city            9 non-null      object
 7   investor_permalink      10 non-null     object
 8   investor_name           10 non-null     object
 9   investor_category_code  7 non-null      object
 10  investor_country_code   8 non-null      object
 11  investor_state_code     8 non-null      object
 12  investor_region         10 non-null     object
 13  investor_city           8 non-null      object
 14  funding_round_type      10 non-null     object
 15  funded_at

In [54]:
#Identify the numeric columns we can represent using more space efficient types.

test_df.select_dtypes(include=['float','integer'])

,raised_amount_usd
0,2000000
1,20000
2,20000
3,20000
4,20000
5,20000
6,20000
7,70000
8,75000
9,100000


In [56]:
int_cols = test_df.select_dtypes(include=['float','integer']).columns

In [57]:
int_cols

Index(['raised_amount_usd'], dtype='object')

In [19]:
# downcast from int64 to int16
#test_df['funded_year'] = pd.to_numeric(test_df['funded_year'],downcast='integer')

In [20]:
# downcast from int64 to int32 possible, maybe not across all chunks
test_df['raised_amount_usd'] = pd.to_numeric(test_df['raised_amount_usd'],downcast='integer')

In [21]:
test_df['raised_amount_usd']

0    2000000
1      20000
2      20000
3      20000
4      20000
5      20000
6      20000
7      70000
8      75000
9     100000
Name: raised_amount_usd, dtype: int32

In [22]:
text_cols = test_df.select_dtypes(include=['object']).columns
text_cols

Index(['company_permalink', 'company_name', 'company_category_code',
       'company_country_code', 'company_state_code', 'company_region',
       'company_city', 'investor_permalink', 'investor_name',
       'investor_category_code', 'investor_country_code',
       'investor_state_code', 'investor_region', 'investor_city',
       'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter'],
      dtype='object')

In [23]:
# For text columns:
# Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.

nuniques_dict = test_df.nunique().to_dict()
#turn some into category dtype
for k,v in nuniques_dict.items():
    if v/len(test_df[k]) < 0.5:
        test_df[k] = test_df[k].astype('category')



In [24]:
#11 objects columns turned into space-efficient dtype 'category'
cat_columns = test_df.select_dtypes(include=['category']).columns

In [25]:
# for col in text_cols:
#     print(test_df[col].value_counts())

In [26]:
no_missing_cols

['company_permalink',
 'company_name',
 'company_country_code',
 'company_region',
 'investor_permalink',
 'investor_name',
 'investor_region',
 'funding_round_type',
 'funded_at',
 'funded_month',
 'funded_quarter',
 'funded_year']

In [27]:
dtype_dict = {t:'category' for t in cat_columns if t in no_missing_cols}

dtype_dict

{'company_country_code': 'category',
 'investor_permalink': 'category',
 'investor_name': 'category',
 'investor_region': 'category',
 'funding_round_type': 'category',
 'funded_year': 'category'}

In [28]:
# Make changes to the code from loading csv so that the overall memory the data consumes stays under 10 megabytes.


In [29]:
test_df.columns[18] # col19 'raised_amount_usd' cannot be "int32" as it has NA
# col18 'raised_year' cannot be "int16" as it has NA

'funded_year'

In [58]:
#use only keep_cols
chunk_iter = pd.read_csv('crunchbase-investments.csv',encoding='latin-1',dtype=dtype_dict,chunksize=5000,usecols=keep_cols)

for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(2**20)) 
#each chunk now consumes around 4.3 mb in memory-- can double up chunksize

3.773927688598633
3.643202781677246
3.647123336791992
3.611940383911133
3.6208715438842773
3.6335649490356445
3.646841049194336
3.633734703063965
3.658021926879883
3.45751953125
1.971994400024414


In [61]:
## best chunksize is 13000 --  so that the overall memory the data consumes stays under 10 megabytes.
chunk_iter = pd.read_csv('crunchbase-investments.csv',encoding='latin-1',dtype=dtype_dict,chunksize=13000,usecols=keep_cols)

for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(2**20)) 

9.607172966003418
9.379837989807129
9.41344165802002
9.562762260437012
0.6198701858520508


In [32]:
#  next step is to load each chunk into a table in a SQLite database so we can query the full data set.

In [63]:
import sqlite3
conn = sqlite3.connect('fundraising.db')

In [ ]:
chunk_iter = pd.read_csv('crunchbase-investments.csv',encoding='latin-1',dtype=dtype_dict,chunksize=13000)

for chunk in chunk_iter:
    #print(chunk.memory_usage(deep=True).sum()/(2**20)) 
    chunk.to_sql('fundraising',conn,if_exists='append',index=False)
    
q = 'PRAGMA table_info(fundraising)' # Query the table and make sure the data types match up to what you had in mind for each column.
pd.read_sql(q,conn)

In [ ]:
q = 'SELECT * FROM fundraising'
sql_iter = pd.read_sql(q,conn,chunksize=500)

In [ ]:
for chunk in sql_iter:
    print(chunk.head(5))
    break

In [ ]:
next(sql_iter)

In [ ]:
!wc -l 'fundraising.db' 
#no. of lines

In [ ]:
#!wc --help
!wc -c 'fundraising.db' 
# byte counts

In [ ]:
10878976/(2**20) #  file size of the database is 10.4mb